In [92]:
import params as par
from background import *

import numba
from scipy.special import spherical_jn
from scipy import integrate
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import copy
import pickle
plt.rcParams.update(par.plot_params)
cp = copy.deepcopy
plt.rcParams.update(par.plot_params)

In [93]:
# Load data from GDM_FID.

τ_solve = pickle.load(open( "data/tau_solve.p", "rb" ) )
k_solve = pickle.load(open( "data/k_solve.p", "rb" ) )

wd_F = pickle.load(open( "data/w_d.p", "rb" ) )
Dwd_F = pickle.load(open( "data/Dw_d.p", "rb" ) )
cs2d_F = pickle.load(open( "data/cs2_d.p", "rb" ) )

δd = pickle.load(open( "data/delta_d.p", "rb" ) )
vd = pickle.load(open( "data/v_d.p", "rb" ) )
δɣ = pickle.load(open( "data/delta_g.p", "rb" ) )
vɣ = pickle.load(open( "data/v_g.p", "rb" ) )
Φ = pickle.load(open( "data/phi.p", "rb" ) )


(wd, Dwd, cs2d) =(wd_F, Dwd_F, cs2d_F)

N = len(τ_solve)

In [100]:
"""1 fluid"""

# set initial conditions
Φ0  = Φ[:,0] 
δɣ0 = δɣ[:,0] 
vɣ0 = vɣ[:,0] 
δd0 = δd[:,0] 
vd0 = vd[:,0] 
Y0 = np.array([Φ0, δɣ0, vɣ0, δd0, vd0])

# define doubly-sampled time array for RK4:
τ_RK = np.interp(np.arange(0, N, 0.5), np.arange(0,N),  τ_solve)


# evaluate functions at solve points
ℋ_RK = 2*α*(α*τ_RK/τr + 1)/(α**2 * (τ_RK**2/τr) + 2*α*τ_RK)
a_RK =  a_eq*((α*τ_RK/τr)**2 + 2*α*τ_RK/τr)
yb_RK = 1.68*a_s*Ωb0/Ωr0
Ωb_RK = Ωb0 * a_s**-3.
Ωɣ_RK = Ωɣ0 * a_s**-4.
Ωd_RK = Ωc0 * a_s**-3. + Ων0 * a_s**-4.
wd_RK = np.interp(τ_RK, τ_solve, wd)
Dwd_RK = np.interp(τ_RK, τ_solve, Dwd)
cs2d_RK = np.array([np.interp(τ_RK, τ_solve, cs2d[i,:]) for i in range(K_SOLVE)])


@numba.jit(nopython=True)
def DY(i, Y, wd, Dwd):
        dY = np.zeros((5,K_SOLVE))
        Φ = Y[0,:]
        δɣ = Y[1,:]
        vɣ = Y[2,:]
        δd = Y[3,:]
        vd = Y[4,:]
           
        #compute background quantities
        ℋi = ℋ_RK[i] 
        ai =  a_RK[i]
        ybi = yb_RK[i]
        Ωbi = Ωb_RK[i]
        Ωɣi = Ωɣ_RK[i]
        Ωdi = Ωd_RK[i]

        wdi = wd_RK[i]
        Dwdi = Dwd_RK[i]
        cs2di = cs2d_RK[:,i]
        
        f = vd*Ωdi*(1+wdi) + 4./3.*Ωɣi*vɣ +  Ωbi*vɣ
        #DΦ
        dY[0,:]  = -ℋi*Φ + (3/2.*H0**2.*ai**2/k_solve)*f
        #Dδɣ
        dY[1,:] = -4./3.*k_solve*vɣ + 4*dY[0,:]
        #Dvɣ
        dY[2,:] = (-ℋi*ybi*vɣ + k_solve*δɣ/3)/(
            4./3. + ybi) + k_solve*Φ
        #Dδd
        dY[3,:] = -(1+wdi)*(k_solve*vd-3*dY[0,:]) -\
            3*ℋi*(cs2di-wdi)*δd
        # Dvd
        dY[4,:] = -ℋi*(1-3*wdi)*vd - vd*Dwdi/(1+wdi) +\
            k_solve*δd*cs2di/(1+wdi) + k_solve*Φ

        return dY
xx = DY(1, Y0, wd_F, Dwd_F)
%prun DY(1, Y0, wd_F, Dwd_F)
%timeit DY(1, Y0, wd_F, Dwd_F)

 2.95 µs ± 80 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [57]:
# compute stepsizes for RK4
stepsizes = np.diff(τ_solve)

# implement RK4
@numba.jit(nopython=True)
def solve(wd=wd_F, Dwd=Dwd_F): 
    Y = np.zeros((len(τ_solve), 5, par.K_SOLVE))
    Y[0,:,:] = Y0
    for i, τi in enumerate(τ_solve[:-1]):
        ss = stepsizes[i]
        k1 = ss*DY(τi,Y[i,:,:], wd, Dwd)
        k2 = ss*DY(τi+ss/2,Y[i,:,:]+k1/2, wd, Dwd)
        k3 = ss*DY(τi+ss/2,Y[i,:,:]+k2/2, wd, Dwd)
        k4 = ss*DY(τi+ss,Y[i,:,:]+k3, wd, Dwd)
        
        Y[i+1,:,:] = Y[i,:,:] + k1/6 + k2/3 + k3/3 + k4/6
    return Y

In [53]:
# k and l list for integration
k_list2  = np.linspace(k_low,k_high,K_INT)
l_list = np.array([i for i in range(2,50,1)]+[i for i in range(50,200,5)] + 
     [i for i in range(200, 1200, 20)])
L = len(l_list)

"""Precompute Bessel Functions"""
# check if they're already stored
if 'JLK' not in globals():
    JLK = np.array([
        spherical_jn(l, k_list2*(τ_now-τ_rec))
                    for l in l_list])
    DJLK = np.array([
        spherical_jn(l, k_list2*(τ_now-τ_rec), derivative=True) 
        for l in l_list])


In [58]:
k_grid = np.broadcast_to(k_list2,(len(l_list),K_INT))
@numba.jit(nopython=True)
def get_Cl(wd = wd_F, Dwd = Dwd_F):
    # first compute mode evolution:
    Yf = solve(wd, Dwd)[-1,:,:]
    SW = Yf[0,:] + Yf[1,:]/4
    DOP = Yf[2,:]
    SWsd = SW*np.exp(-(k_solve*τs)**2)
    DOPsd = DOP*np.exp(-(k_solve*τs)**2)
    
    # get the power spectrum
    SWfill = np.interp(k_list2, k_solve, SW)
    DOPfill = np.interp(k_list2, k_solve, DOP)
    Dl = SWfill*JLK + DOPfill*(DJLK-JLK/(2*k_list2*(τ_now-τ_rec)))

    T = np.exp(-2*(k_list2*τs)**2 - (.03*k_list2*τ_rec)**2)
    Cl_itgd = Dl**2 *T /k_list2
    Cl = integrate.trapz(k_grid, Cl_itgd)
    Cl_out = l_list*(l_list + 1)*Cl
    return -Cl_out

In [59]:
# compile the functions and time
time_functions=True
solve()
get_Cl()
if time_functions:
    print('timing mode evolution...')
    %timeit solve()
    print('\n')
    print('timing power spectrum computation...')
    %timeit get_Cl()


timing mode evolution...
20 ms ± 274 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


timing power spectrum computation...
53 ms ± 521 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
"""w-only PCA"""

# number of components:
N_C = 600
J = np.zeros((L, N_C))

# derivative stepsize:
stepsize = 0.00001

# set the w basis
basis_domain = np.linspace(τ0, τ_rec, 4*N_C)
B = np.zeros((len(basis_domain),N_C))
# sin/cosine basis
for n in range(N_C//2):
    s_n = np.sin((n+1)*np.pi*basis_domain/τ_rec)
    c_n = np.cos(n*np.pi*basis_domain/τ_rec)
    B[:,2*n] = s_n
    B[:,2*n+1] = c_n


# basis coefficients:
Θw = np.zeros(N_C)

# standard basis vectors:
e = np.eye(N_C)

# Calculate the fiducial power spectrum
wd = cp(wd_F)
Dwd = cp(Dwd_F)
Cl0 = get_Cl()

In [61]:
# Compute the partials:
for n in range(N_C):
    global wd, Dwd
    # perturb w forward and backward
    Θw = e[n] * stepsize
    step = np.interp(τ_solve, basis_domain, B@Θw)
    wd_fwd = wd_F + step
    wd_bwd = wd_F - step
    
    # calculate Dw
    Dwd_fwd = np.gradient(wd_fwd, τ_solve)
    Dwd_bwd = np.gradient(wd_bwd, τ_solve)
    
    # evaluate Cl
    (wd, Dwd) = (wd_fwd, Dwd_fwd)
    Cl_fwd = get_Cl()
    (wd, Dwd) = (wd_bwd, Dwd_bwd)
    Cl_bwd = get_Cl()
    J[:,n] = (Cl_fwd - Cl_bwd)/(2.*stepsize) 


KeyboardInterrupt: 

In [ ]:
print(np.shape(J))
#compute Fisher matrix
F = np.transpose(J)@np.eye(L)@J
plt.imshow(J)
plt.show()
plt.imshow(F)
plt.show()

In [ ]:
w, v = np.linalg.eig(F)
PC = np.real(v)
plt.imshow(PC)
plt.show()
plt.semilogy(np.abs(w))
plt.title('eigenvalues')
plt.show()

In [ ]:
print(np.shape(PC))
PF = B@PC
plt.plot(basis_domain,PF[:,0:5])
plt.xlim(-.000005,0.0002)
plt.show()
plt.semilogy(np.abs(w)[0:10])